# Tiny Generative Pretrained Transformer

Generating infinite Shakespeare by training a transformer based language model on all of Shakespeare's work. 

Lots of help from Arxiv and Andrej Karpathy

In [37]:
import numpy as np
from matplotlib.pyplot import plot as plt
import torch

### Links

Annotated Transformer - http://nlp.seas.harvard.edu/annotated-transformer/

The Illustrated Tranformer - https://jalammar.github.io/illustrated-transformer/





### Dataset

Tiny models need tiny datasets. 

We are using the Tiny Shakespeare dataset, which is a concatenation of all the works of Shakespeare - https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt

In [38]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-02-05 20:38:35--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2023-02-05 20:38:36 (11.7 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [39]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [40]:
print(f"Number of characters: {len(text)}") # over a million characters!

Number of characters: 1115394


In [41]:
# first 1000 words
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [42]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Characters in set:" + ''.join(chars))
print(f"Vocab size: {vocab_size}")

Characters in set:
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


### Tokenization

Real GPT's use tiktoken - https://github.com/openai/tiktoken

Below is just a character level tokenizer

In [43]:
# translating characters into integers

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]          # encoder: String -> List[Ints]
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: List[Ints] -> String

In [44]:
encode("Hello world!")

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]

In [45]:
decode([20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2])

'Hello world!'

In [46]:
# Encoding the entire Shakespeare work

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### Train and Validation Splits


In [47]:
# Make 90% of the dataset to be train with 10% validation 
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

Putting the entire chuck of text into the transformer at once in computationally infeasible. So, we need to sample little chunks of the dataset, and feed that into the transformer.

In [48]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [49]:
x = train_data[:block_size] # inputs to the transformer
y = train_data[1:block_size+1] # targets for each position, offset by one 

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the input is {context}, the target is: {target}")

When the input is tensor([18]), the target is: 47
When the input is tensor([18, 47]), the target is: 56
When the input is tensor([18, 47, 56]), the target is: 57
When the input is tensor([18, 47, 56, 57]), the target is: 58
When the input is tensor([18, 47, 56, 57, 58]), the target is: 1
When the input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [50]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [51]:
torch.manual_seed(1337)

batch_size = 4 # how many sequences we process every forward and backwards pass
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split=='train' else val_data
    
    # randomly get indexes of dataset
    idx = torch.randint(len(data) - block_size, (batch_size,))
    
    # torch.stacks get the one dimensional tensor like [18, 47, 56, 57, 58,  1, 15, 47]
    # and puts them all together into a 4x8 tensor
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x.to(device),y.to(device)
    

In [52]:
x_batched, y_batched = get_batch('train')

print(f"inputs: {x_batched.shape}") # a 4x8 array contains 32 indepenednt examples, each shown in new cell
print(x_batched)
print(f"targets: {y_batched.shape}")
print(y_batched)
print('----')

inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='mps:0')
targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='mps:0')
----


In [53]:
for batch in range(batch_size):
    for t in range(block_size):
        context = x_batched[batch, :t+1]
        target = y_batched[batch,t]
        
        # all of these are independent, as far as the transformer is concerned
        print(f"When input is {context.tolist()}, the target is {target}")

When input is [24], the target is 43
When input is [24, 43], the target is 58
When input is [24, 43, 58], the target is 5
When input is [24, 43, 58, 5], the target is 57
When input is [24, 43, 58, 5, 57], the target is 1
When input is [24, 43, 58, 5, 57, 1], the target is 46
When input is [24, 43, 58, 5, 57, 1, 46], the target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
When input is [44], the target is 53
When input is [44, 53], the target is 56
When input is [44, 53, 56], the target is 1
When input is [44, 53, 56, 1], the target is 58
When input is [44, 53, 56, 1, 58], the target is 46
When input is [44, 53, 56, 1, 58, 46], the target is 39
When input is [44, 53, 56, 1, 58, 46, 39], the target is 58
When input is [44, 53, 56, 1, 58, 46, 39, 58], the target is 1
When input is [52], the target is 58
When input is [52, 58], the target is 1
When input is [52, 58, 1], the target is 58
When input is [52, 58, 1, 58], the target is 46
When input is [52, 58, 1, 58, 46

### Bigram Language Model

The most simple baseline.

A good loss function is negatively log likelihood loss aka in PyTorch as ```cross_entropy()```

In [54]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [55]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx and targets are both (Batch, Time) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel) 
        
        if targets is None:
            loss = None
        else: 
            # PyTorch expects (B, C, T), we need to convert
            B, T, C = logits.shape

            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # get the predictions
            
            # focus on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) 
            
            # sample from the distribition
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [56]:
model_0 = BigramLanguageModel(vocab_size).to(device)
logits, loss = model_0(x_batched, y_batched)
print(f" logits shape: {logits.shape}, loss: {loss}")

 logits shape: torch.Size([32, 65]), loss: 4.878634452819824


In [57]:
idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model_0.generate(idx, max_new_tokens=100)[0].tolist()))


SjbhKptLNsA:q,kSPw'aPGXE$i
SpxnkXUzkzC-wJr.

;u!Idl,obcHA-uQ'
b3sZ 3KUD&j3Sq-'d'qb?Elnlh'nFbEQbduehT


### Training Bigram Model

In [58]:
optimizer = torch.optim.AdamW(model_0.parameters(), lr=1e-3)

In [59]:
batch_size = 32

for steps in range(10000):
    # sample a batch of data
    x_batched, y_batched = get_batch('train')
    
    # evaluate the loss
    logits, loss = model_0(x_batched, y_batched)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.394822120666504


In [60]:
print(decode(model_0.generate(idx, max_new_tokens=300)[0].tolist()))



Houreng; wou ino clY kn po thace;
CETonourd? s llly s, ind anr JMESck der ait We we bueswars
Pis ftw$VOH:
ro le it h y,

OULINBee, bysas w OLLORLonea merond s teathist storir; ms hen,-ovy isowisthis thay lororeilar tho thothatiory ad Is arourst masund m mpuld t ng heaie.
RRer t t howoo nd saresquet


It's looks like it is kind of working!?

### Self attention

Tokens should not be able to use future tokens as context.

So, we want to calculate the average of all vectors in all the previous n tokens, including n. We can use this using a bag of words https://en.wikipedia.org/wiki/Bag-of-words_model



In [77]:
torch.manual_seed(1337)
B, T, C = 4, 8 , 32

x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 32])

In [78]:
x_bag_of_words = torch.zeros(B,T,C)

for b in range(B):
    for t in range(T):
        x_prev = x[b,:t+1] # (t,C)
        x_bag_of_words[b,t] = torch.mean(x_prev, 0)

The first row of these are the same, which makes sense because we have only looked at one word. The next rows change as we are then taking the average of 0...n.

In [79]:
x[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-6.6310e-01, -2.5128e-01,  1.0101e+00,  1.2155e-01,  1.5840e-01,
          1.1340e+00, -1.1539e+00, -2.9840e-01, -5.0754e-01, -9.2392e-01,
          5.4671e-01, -1.4948e+00, -1.2057e+00,  5.7182e-01, -5.9735e-01,
         -6.9368e-01,  1.6455e+00, -8.0299e-01,  1.3514e+00, -2.7592e-01,
         -1.5108e+00,  2.1048e+00,  2.7630e+00, -1.7465e+00,  1.4516e+00,
         -1.5103e+00,  8.2115e-01, -2.1153e-01,  7.7890e-01,  1.5333e+00,
          1.6097e+00, -4.0323e-01],
        [-8.3447e-01,  5.9780e-01, -5.14

In [80]:
x_bag_of_words[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-2.4116e-01, -1.6063e-01,  3.2526e-01, -3.9683e-01,  3.9208e-01,
          5.7976e-01, -9.9692e-02, -1.1702e-01, -7.3193e-02,  1.2198e-01,
         -4.0159e-01, -1.0025e+00, -4.8488e-01,  1.6602e-01, -7.5923e-01,
          4.2481e-01,  1.4972e+00, -4.7132e-01,  8.1860e-01,  3.4460e-01,
         -1.7740e+00,  1.2990e+00,  2.1250e+00, -5.7775e-01,  7.8881e-01,
         -1.5365e+00, -1.6947e-01, -2.7318e-01,  6.1334e-01,  3.6581e-01,
          1.5667e+00,  1.0527e+00],
        [-4.3893e-01,  9.2179e-02,  1.99

That was inefficient, can make it much better using matrix multiplication. More specifically, we can use ```torch.tril()``` to create a lower triangular matrix. This is the trick to block out the future tokens from the current token. 

In [81]:
weights = torch.tril(torch.ones(T,T))
weights = weights / weights.sum(1, keepdim=True)
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [82]:
# batched matrix multiply
# shapes are not the same, pytorch adds batch dimension
# original: (T,T) @ (B,T,C) --> ???
# updated:(B,T,T) @ (B,T,C) --> (B,T,C)
x_bag_of_words_2 = weights @ x 
x_bag_of_words_2

tensor([[[ 1.8077e-01, -6.9988e-02, -3.5962e-01,  ..., -8.0164e-01,
           1.5236e+00,  2.5086e+00],
         [-2.4116e-01, -1.6063e-01,  3.2526e-01,  ...,  3.6581e-01,
           1.5667e+00,  1.0527e+00],
         [-4.3893e-01,  9.2179e-02,  1.9971e-01,  ...,  9.8210e-02,
           7.1071e-01,  5.6531e-01],
         ...,
         [-9.8921e-01,  1.3417e-01,  2.8014e-01,  ...,  3.4950e-01,
           6.1414e-01,  1.2510e-01],
         [-8.2062e-01,  6.6077e-02,  4.9662e-01,  ...,  3.5242e-01,
           4.4703e-01,  5.0332e-02],
         [-7.9077e-01,  3.0213e-02,  4.3624e-01,  ...,  6.9874e-02,
           3.2521e-01,  1.7912e-01]],

        [[ 4.5618e-01, -1.0917e+00, -8.2073e-01,  ...,  5.1187e-02,
          -6.5764e-01, -2.5729e+00],
         [ 2.3859e-01, -4.2831e-02, -1.0349e+00,  ...,  4.1852e-01,
          -9.0388e-01, -6.2984e-01],
         [ 8.9262e-01, -1.0170e-01, -5.0905e-01,  ...,  6.4184e-02,
          -2.4146e-01, -6.8638e-01],
         ...,
         [ 5.6778e-01,  3

In [83]:
# x_bag_of_words and x_bag_of_words_2 are the same now
# Using the matrix way is significantly faster, however
torch.allclose(x_bag_of_words, x_bag_of_words_2)

False

However, this is the best way to do the lower triangular batched matrix multiplication:

In [84]:
tril = torch.tril(torch.ones(T,T))
weights = torch.zeros((T,T))
weights

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [85]:
# make all zero weights negative infinity
weights = weights.masked_fill(tril == 0, float('-inf'))
weights

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

### Aha-moment:
This is super useful in self attention, because this weights matrix tells us how much of each vector to pay attention to! We can train this to get specialized values, but also making sure the future can't communicate the past.

In [86]:
# We then take softmax() along every row
# https://en.wikipedia.org/wiki/Softmax_function
# This is just normalization, which creates the weight matrix

weights = F.softmax(weights, dim=-1)
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [87]:
# The final bag of words
out = weights @ x

In [88]:
out.shape

torch.Size([4, 8, 32])

### Crux of Self Attention

We should just use uniform weights, each equal and add up to one in our weights matrix. Some words want to pay more attention to others, such as how a vowel might pay attention to different consonants with differing weights. We still want to gather information from the past, but in a data dependent way. (Encoders don't use ```torch.tril()```, decoders do)

How does self attention solve this?
Every single token at each position emits two vectors:
1. Query
2. Key

What does the query vector do?
It represents what we are looking for. Like a google search query or a SQL query.

That does the key vector do?
It represents what the weights contain.

To find the similarity, or affinity between these two vectors, we simply do a dot product between the keys and the queries. If the key and query are aligned, then they interact a very high amount, and it pays more attention to that specific token. 

We can do that using a single self-attention head.


In [98]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)


k = key(x)                       # (B,T,16)
q = query(x)                     # (B,T,16)
weights = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
# weights = torch.zeros((T,T))
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=-1)

v = value(x)
out = weights @ v

out.shape

torch.Size([4, 8, 16])

In [97]:
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [99]:
from tqdm import tqdm